In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1.Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2.データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ：\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ：
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3.OpenAI APIの呼出

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text}
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

売上データからいくつかの重要な傾向を分析するために、以下の観点からデータを整理して考察します。

### 1. カテゴリー毎の売上分析
- **食品、衣服、日用品の各カテゴリー**の売上を集計して、それぞれの貢献度を比較します。
- **売上高**は単価×数量で計算し、利益（売上高−原価）も計算します。

```python
import pandas as pd

# 売上データのサンプル作成
data = {
    'カテゴリー': ['食品', '食品', '食品', '衣服', '衣服', '衣服', '衣服', '食品', '日用品', '衣服', '食品'],
    '商品コード': [1001, 1002, 1003, 2001, 2002, 2077, 2078, 1085, 3077, 2078, 1086],
    '商品名': ['りんご', 'バナナ', '牛乳', 'Tシャツ', 'ジーンズ', 'レインパンツ', 'レインシューズ', 'ザクロ', 'バスブラシ', 'レインシューズ', 'ココナッツ'],
    '売上日': pd.date_range(start='2023-01-01', periods=10, freq='D'),
    '単価': [200, 150, 180, 1500, 5000, 2000, 2500, 600, 400, 300],
    '数量': [50, 100, 80, 20, 10, 18, 15, 40, 60, 15],
    '原価': [120, 80, 100, 800, 2500, 1000, 1250, 300, 200, 150]
}

df = pd.DataFrame(data)
df['売上高'] = df['単価'] * df['数量']
df['利益'] = df['売上高'] - df['原価']
```

### 2. カテゴリー別の総売上および利益の計算
以下の計算を行うことで、各カテゴリーの売上と利益の合計を把握できます。

```python
category_summary = df.groupby('カテゴリー').agg({'売上高':'sum', '利益':'sum'})
```

### 3. 時系列分析
- 売上の日付に基づいて、月毎

In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split('\n')
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0     売上データからいくつかの重要な傾向を分析するために、以下の観点からデータを整理して考察します。
1                                                    
2                                  ### 1. カテゴリー毎の売上分析
3      - **食品、衣服、日用品の各カテゴリー**の売上を集計して、それぞれの貢献度を比較します。
4               - **売上高**は単価×数量で計算し、利益（売上高−原価）も計算します。
5                                                    
6                                           ```python
7                                 import pandas as pd
8                                                    
9                                      # 売上データのサンプル作成
10                                           data = {
11      'カテゴリー': ['食品', '食品', '食品', '衣服', '衣服', '衣...
12      '商品コード': [1001, 1002, 1003, 2001, 2002, 20...
13      '商品名': ['りんご', 'バナナ', '牛乳', 'Tシャツ', 'ジーンズ'...
14      '売上日': pd.date_range(start='2023-01-01', p...
15      '単価': [200, 150, 180, 1500, 5000, 2000, 25...
16      '数量': [50, 100, 80, 20, 10, 18, 15, 40, 60...
17      '原価': [120, 80, 100,

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel('売上データ分析結果.xlsx', index=False)